In [1]:
import plotly.graph_objects as go
from dataTable import *
from weaponData import *

generalSettings = et.parse("settings.xml").getroot().find("general")
health = int(generalSettings.get("health"))
interpolation = generalSettings.get("interpolation")
def plotWeapon(weaponToPlot, xAxis = 'distance', yAxis = 'TTK', hoverFields = ['STK', 'damage', 'fireInterval', 'fireRate']):
    global fig
    id = 0
    try:
        id = int(weaponToPlot)
    except:
        # 把weaponToPlot从武器数据转换到ID
        id = getIDByName(weaponToPlot)
    weapon = WeaponData(id)
    killData = weapon.getKillData(health)
    x=killData[xAxis].tolist()
    y=killData[yAxis].tolist()
    hoverText = []
    # 遍历X轴的所有数据
    for i in x:
        text = ''
        # 找到浮动窗口中对应当前X的数值
        for j in hoverFields:
            text += j + ': ' + str(killData[killData[xAxis] == i][j].iloc[0]) + '<br>'
        hoverText.append(text)
    fig.add_trace(go.Scatter(x=x, y=y, line_shape = interpolation, name = weapon.name + ": {}".format(str(weapon.id)),
                             hovertemplate = 
                                 '<br><b>' + yAxis + ': %{y}</b><br>' + 
                                 '<b>' + xAxis + ': %{x}</b><br>' +
                                 '%{text}', 
                                 text = hoverText,
                             ))
    fig.update_layout(hovermode='x unified', xaxis_title=xAxis, yaxis_title=yAxis, width=1000, height=600,)

In [2]:
fig = go.FigureWidget()
# 在此声明要绘制哪些武器
plotWeapon('M4')
fig

FigureWidget({
    'data': [{'hovertemplate': '<br><b>TTK: %{y}</b><br><b>distance: %{x}</b><br>%{text}',
              'line': {'shape': 'vh'},
              'name': 'M4: 0',
              'text': [STK: 4.0<br>damage: 30.0 (30.0 * 1)<br>fireInterval:
                       0.08<br>fireRate: 750.0<br>, STK: 4.0<br>damage: 30.0 (30.0
                       * 1)<br>fireInterval: 0.08<br>fireRate: 750.0<br>, STK:
                       5.0<br>damage: 20.0 (20.0 * 1)<br>fireInterval:
                       0.08<br>fireRate: 750.0<br>, STK: 7.0<br>damage: 16.0 (16.0
                       * 1)<br>fireInterval: 0.08<br>fireRate: 750.0<br>],
              'type': 'scatter',
              'uid': '61e31b89-038d-4416-94a2-41661c800202',
              'x': [0.0, 10.0, 25.0, 50.0],
              'y': [0.24, 0.24, 0.32, 0.48]}],
    'layout': {'height': 600,
               'hovermode': 'x unified',
               'template': '...',
               'width': 1000,
               'xaxis': {'title': {'t